In [ ]:
import numpy as np
from tensorflow import keras
import tensorflow as tf
import pandas as pd


from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!pip install patool
#import patoolib
#patoolib.extract_archive("/content/drive/MyDrive/meuseum/museum_exhibits_cv_hack.rar", outdir="/content/drive/MyDrive/meuseum/")

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/meuseum/train_dataset_train/train.csv")

In [ ]:
df_train.head()

,id,description,object_img
0,520,Фотография. Г. Пермь. Здание горисполкома. ПО...,799
1,1817,Фотонегатив пленочный. Труппа театра «У моста»...,854
2,188,"Изразец гладкий расписной ""пермский""- карниз А...",1794
3,1472,Фотонегатив пленочный. Серебряная чаша из Хоре...,526
4,1664,Фотонегатив стеклянный. Этнография коми-пермяк...,244


In [ ]:
df_train.index = df_train['id']

In [ ]:
df_train.loc[[520, 1817]]

,id,description,object_img
id,,,
520,520,Фотография. Г. Пермь. Здание горисполкома. ПО...,799
1817,1817,Фотонегатив пленочный. Труппа театра «У моста»...,854


In [ ]:
df_train['filename'] = df_train['object_img'].apply(lambda x: f"/content/drive/MyDrive/meuseum/train_dataset_train/train/{x}.png")

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


train_generator = train_datagen.flow_from_dataframe(
    df_train,
    y_col='id',
    target_size=(32, 32),
    batch_size=32,
    class_mode='raw')

Found 2098 validated image filenames.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cnt = CountVectorizer(min_df=0.07)
cnt.fit_transform(df_train['description'])

<2098x51 sparse matrix of type '<class 'numpy.int64'>'
	with 15813 stored elements in Compressed Sparse Row format>

In [ ]:
class TrainDataGenerator(Sequence):
    """Generates data for Keras
    Sequence based data generator. Suitable for building data generator for training and prediction.
    """
    def __init__(self, df, vectorizer, to_fit=True, batch_size=32, dim=(150, 150),
                 n_channels=3, shuffle=True):
        self.train_datagen = ImageDataGenerator(
                                  rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)
        self.train_generator = train_datagen.flow_from_dataframe(
                                  df_train,
                                  y_col='id',
                                  target_size=(32, 32),
                                  batch_size=32,
                                  class_mode='raw')
        self.vectorizer = vectorizer
        self.df = df


    def __len__(self):
        """Denotes the number of batches per epoch
        :return: number of batches per epoch
        """
        return len(self.train_generator)

    def __getitem__(self, index):
        X, y_idx = self.train_generator[index]
        rows = self.df.loc[y_idx]
        y = self.vectorizer.transform(rows['description']).todense()
        return X, y


    def on_epoch_end(self):
        pass


In [ ]:
data_generator = TrainDataGenerator(df_train, cnt)

Found 2098 validated image filenames.


In [ ]:
input_shape = (32, 32, 3)
num_features = len(cnt.vocabulary_)
keras_tf_layers = tf.keras.layers

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        keras_tf_layers.Conv2D(32, kernel_size=(4, 4), activation="relu"),
        keras_tf_layers.MaxPooling2D(pool_size=(2, 2)),
        keras_tf_layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras_tf_layers.MaxPooling2D(pool_size=(2, 2)),
        keras_tf_layers.Flatten(),
        keras_tf_layers.Dropout(0.5),
        keras_tf_layers.Dense(num_features, activation="sigmoid"),
    ]
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mae"])

In [ ]:
model.fit(data_generator, epochs=100)

In [ ]:
model.save('/content/drive/MyDrive/meuseum/model2.h5')

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/meuseum/model2.h5')

In [ ]:
df_test = pd.read_csv("/content/drive/MyDrive/meuseum/test_dataset_test/test.csv")

In [ ]:
df_test.head()

,id,description
0,486,Фотография. Елизавета Алексеевна Юманова. ПКМ...
1,813,Фотография. Заседание комитета комсомола мотор...
2,2980,"Фотография. День ""Саланга"". ПОКМ-18530/638 фо..."
3,13,Фотография. Елизавета Алексеевна Юманова. ПКМ...
4,2467,"Фотография. ""Универмаг Пермь. Оформление демон..."


In [ ]:
import os
filenames = os.listdir("/content/drive/MyDrive/meuseum/test_dataset_test/test")

test_df_img = pd.DataFrame({'id': list(range(len(filenames))),
                            'filename': list(map(lambda x: f"/content/drive/MyDrive/meuseum/test_dataset_test/test/{x}", filenames))})

In [ ]:
test_datagen = ImageDataGenerator(
    rescale=1./255)


test_generator = test_datagen.flow_from_dataframe(
    test_df_img,
    y_col='id',
    target_size=(32, 32),
    class_mode='raw',
    shuffle=False)


NameError: ignored

In [ ]:
raw_predictions = model.predict(test_generator)

In [ ]:
raw_predictions[0]

In [ ]:
test_vectors = cnt.transform(df_test['description']).todense()

In [ ]:
res_map = {}

for i in range(len(raw_predictions)):
  dist = np.inf
  txt_n = None
  for j in range(len(test_vectors)):
    if j in res_map.values():
      continue
    
    dist_ = np.sum(np.abs(raw_predictions[i] - test_vectors[j]))
    if dist_ < dist:
      txt_n = j
      dist = dist_

  res_map[i] = txt_n

In [ ]:
res_imgs = [filenames[y[0]] for y in sorted(res_map.items(), key=lambda x: x[1])]

In [ ]:
submission = pd.DataFrame({'id': df_test['id'], 'object_img': list(map(lambda x: x.split('.')[0], res_imgs))})

In [ ]:
submission

,id,object_img
0,486,1483
1,813,521
2,2980,1624
3,13,2990
4,2467,789
...,...,...
895,2860,928
896,2551,2149
897,2399,944
898,1492,1957


In [ ]:
submission.to_csv('/content/drive/MyDrive/meuseum/submission3.csv', index=False)

In [ ]:
pd.concat([df_train, df_test]).sort_values(by='object_img')

,id,description,object_img
1408,725,Монета ДМ/Н-1104 ПОКМ-3873/6 серебро; штамповк...,1.0
2075,134,Монета ДМ/Н-1101 ПОКМ-3873/3 серебро; штамповк...,2.0
1083,1617,Монета ДМ/Н-1100 ПОКМ-3873/2 серебро; штамповк...,3.0
385,646,Монета ДМ/Н-1103 ПОКМ-3873/5 серебро; штамповк...,4.0
462,705,Монета ДМ/Н-850 ПОКМ-11035/17 серебро; штампов...,5.0
...,...,...,...
895,2860,Фотонегатив пленочный. Делегаты VI съезда комп...,NaN
896,2551,"Изразец печной ПОКМ-20318/52 формовка,глина (...",NaN
897,2399,"Фотография. ""Универмаг Пермь. Оформление демон...",NaN
898,1492,ПЗС. Подвеска шумящая коньковая ПКМ-21222/274...,NaN


In [ ]:
train_vectors = cnt.transform(df_train['description']).todense()
test_vectors = cnt.transform(df_test['description']).todense()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarities = cosine_similarity(train_vectors, test_vectors)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


In [ ]:
similar_train_idx = np.argmax(similarities, axis=0)

In [ ]:
similar_train_idx

array([ 211, 1224, 1549,  211,  565,  946,  622,   90, 1532, 1035, 1101,
        799,  615,   12, 1482, 1025, 1518,  124,  451,  504,   94,  870,
       2030,   15,   15,  108,   15,  809,   15,  172, 1082,   13,  713,
         15,   43, 1573,    0, 1793,   21,  248,    0,  130,  713,   13,
         15, 1204,   43,  878, 1883,  775,  398, 2053,  754,  898,   15,
        261,   15, 1936,   15,  136,  543,  688,   15,   43, 1131,   38,
       1213, 1566,  875,  789,    5,    0,  328,  485,  903,  396,  238,
       1025, 1657, 1793,   43,  932,  434,   20,   15,  427,  630,   38,
         51, 2072, 1010, 2041,    1, 1903, 1033, 1217,  347,  379,   43,
        525, 1589,  496,  108,    0, 1585,  227,  520,  135,  283, 1532,
         55,  174, 1131, 1346,  130,   45,  698, 1769, 1641, 1720,  458,
       1939,   38,  525, 1634, 1465, 1921,  289,   43, 1024, 1971, 1306,
       1842,    0,  294,   38, 1487,  134, 2083,  331,   15,  913,    0,
        920,  499,  754,  131,  211,   15,   43,   

In [ ]:
import os
test_labels = [int(x.split('.')[0]) for x in os.listdir("/content/drive/MyDrive/meuseum/test_dataset_test/test")]

In [ ]:
train_labels = [int(x) for x in df_train['object_img']]

In [ ]:
mapping = {}

for i, sim in enumerate(similar_train_idx):
  target_train = train_labels[sim]
  for j in range(1, 500):
    if target_train + j not in mapping:
      mapping[target_train + j] = i
      break
    if target_train - j not in mapping:
      mapping[target_train - j] = i
      break



In [ ]:
result = [x[1] for x in sorted({v: k for k, v in mapping.items()}.items())]

In [ ]:
submission = pd.DataFrame({'id': df_test['id'], 'object_img': result})

In [ ]:
submission.to_csv('/content/drive/MyDrive/meuseum/submission_02.csv', index=False)